In [1]:
import matplotlib.pyplot as plt
import torch
#from sklearn.manifold import TSNE
from torch_geometric.nn import Node2Vec
import pandas as pd
import glob

2024-11-27 19:32:32.726207: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 19:32:32.879061: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Paths to get nodes and edges

Asked ChatGPT: I created a csv in spark that created multiple csv. How can I read them together on pandas?

In [3]:
path_edges = "edges_network3/"
# Use glob to find all the part files
path_edges = glob.glob(path_edges + "part-*.csv")

nodes_path = 'nodes_network3/nodes.csv'

Create graph from data, following Pytorch geometric documentation: https://pytorch-geometric.readthedocs.io/en/latest/tutorial/load_csv.html

First, load nodes into pytorch objects

In [4]:
#Function from documentation
def load_node_parquet(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [5]:
#Class from documentation
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [6]:
words_x, words_mapping = load_node_parquet(nodes_path, 
                                           index_col='node',
                                           encoders={'emo_pos': IdentityEncoder(dtype=torch.long),
                                                    'emo_neg': IdentityEncoder(dtype=torch.long),
                                                    'emo_anx': IdentityEncoder(dtype=torch.long),
                                                    'emo_anger': IdentityEncoder(dtype=torch.long),
                                                    'emo_sad': IdentityEncoder(dtype=torch.long),
                                                    'moral': IdentityEncoder(dtype=torch.long)})
                                               

In [7]:
words_x

tensor([[0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0],
        ...,
        [1, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0]])

Now, create edges

In [8]:
#Function from documentation, modified to read multiple csv
def load_edge_csv(src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = pd.concat((pd.read_csv(f, **kwargs) for f in path_edges))

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

In [9]:
edge_index, edge_label = load_edge_csv(
    src_index_col='node1_norm',
    src_mapping=words_mapping,
    dst_index_col='node2_norm',
    dst_mapping=words_mapping,
    encoders={'weight': IdentityEncoder(dtype=torch.long)},
)

In [10]:
edge_index

tensor([[657, 790, 700,  ..., 597, 470, 530],
        [683, 198,  88,  ..., 530,  94, 230]])

In [11]:
edge_label

tensor([[ 3],
        [14],
        [ 2],
        ...,
        [ 1],
        [ 1],
        [ 1]])

In [12]:
from torch_geometric.utils import to_undirected

undirected_edge_index = to_undirected(edge_index)

In [13]:
from torch_geometric.data import Data

data = Data(x=words_x, edge_index=undirected_edge_index)

In [14]:
data

Data(x=[803, 6], edge_index=[2, 437740])

In [15]:
data.is_directed()

False

## Train Node2Vec model

In [16]:
# Train on GPU if available; takes <=1 min on GPU, ~15 min on CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize Node2Vec model with hyperparameters
model = Node2Vec(
    data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,
    q=1.0,
    sparse=True,
).to(device)

# define data loader (generates batches of rand walks) and optimizer
# for training Node2Vec model:
loader = model.loader(batch_size=128, shuffle=True)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

# Train for 100 epochs and print progress
for epoch in range(1, 101):
    loss = train()
    acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: {acc:.4f}')

ImportError: 'Node2Vec' requires either the 'pyg-lib' or 'torch-cluster' package